In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("/Users/guapofish/Desktop/State_street_test.xlsx")
df.head()

,ISIN,Rating,Coupon,Maturity Date,Yield,EAD
0,US001055AD49,BAA1,3.75,2025-07-10,3.122,1.298000e+09
1,US001055AM48,BAA2,4.75,2021-11-15,2.293,1.396000e+09
2,US001055AN21,A3,3.75,2021-04-15,1.750,5.700000e+08
3,US001055AP78,A3,4.75,2021-01-12,2.332,2.680000e+08
4,US0010EPAF55,A1,1.90,2021-09-15,1.854,4.206000e+09


# 1. Write code to conduct data cleaning (Clean NA)

In [3]:
print(df.isnull().sum())
print("The num of row of the data frame is " +str(len(df)))

ISIN             0
Rating           4
Coupon           3
Maturity Date    3
Yield            4
EAD              1
dtype: int64
The num of row of the data frame is 2467


In [4]:
# Yield/Coupon: fill the NA values by its industry average
df["Coupon"] = df["Coupon"].replace(np.nan,df["Coupon"].mean())
df["Yield"] = df["Yield"].replace(np.nan,df["Yield"].mean())

# Maturity Date/Rating： fill the NA values by its industry mode
mode_rating = df['Rating'].value_counts().index[0]
df["Rating"] = df["Rating"].replace(np.nan,mode_rating)
mode_maturity = df['Maturity Date'].value_counts().index[0]
df['Maturity Date'] = df['Maturity Date'].replace(np.nan,mode_maturity)

# EAD: delete NA values
df['EAD'] = df['EAD'].dropna()

In [5]:
# check the new df
print(df.isnull().sum())
print("The num of row of the data frame is " +str(len(df)))

ISIN             0
Rating           0
Coupon           0
Maturity Date    0
Yield            0
EAD              1
dtype: int64
The num of row of the data frame is 2467


In [6]:
# round up the quarter to maturity for later uses
now = '12/31/2019'
now = pd.to_datetime(now)
df['Quarter to Maturity'] = round(np.subtract(pd.to_datetime(df['Maturity Date']),
                                              now).astype('timedelta64[D]')/91)
df.head()

,ISIN,Rating,Coupon,Maturity Date,Yield,EAD,Quarter to Maturity
0,US001055AD49,BAA1,3.75,2025-07-10,3.122,1.298000e+09,22.0
1,US001055AM48,BAA2,4.75,2021-11-15,2.293,1.396000e+09,8.0
2,US001055AN21,A3,3.75,2021-04-15,1.750,5.700000e+08,5.0
3,US001055AP78,A3,4.75,2021-01-12,2.332,2.680000e+08,4.0
4,US0010EPAF55,A1,1.90,2021-09-15,1.854,4.206000e+09,7.0


The tab “TransitionProb” contains quarterly rating transition probabilities from 2020Q1 to 2020Q4. For each from_bucket (1-8), there should be 8 possible rating transition outcomes (i.e., 8 to_bucket). As such, for each quarter and each from_bucket, the sum of the transition probabilities cross all transitions should be 1. 

(1) check all of the probabilities range correctly between 0 and 1. <br/>
(2) check the sum of transition probabilities for each quarter and each from_bucket equals to 1;

In [7]:
TransitionProb = pd.read_excel("/Users/guapofish/Desktop/State_street_test.xlsx", sheet_name ="TransitionProb")
TransitionProb.head()

,Quarter,From_Bucket,To_Bucket,Probability
0,2020Q1,1,1,0.9905
1,2020Q1,1,2,0.0095
2,2020Q1,1,3,0.0000
3,2020Q1,1,4,0.0000
4,2020Q1,1,5,0.0000


In [8]:
# check all of the probabilities range correctly between 0 and 1
temp = TransitionProb[TransitionProb['Probability'] >= 0]
temp = temp[temp['Probability'] <= 1]
print("The statement of the probabilities range correctly between 0 and is "+ str((len(temp) - len(TransitionProb)) == 0))

The statement of the probabilities range correctly between 0 and is False


In [9]:
#print the false row 
TransitionProb[TransitionProb['Probability'] <0]

,Quarter,From_Bucket,To_Bucket,Probability
207,2020Q4,2,8,-0.007929


In [10]:
# check the sum of transition probabilities for each quarter and each from_bucket equals to 1
print('All the sum of transition probabilities for each quarter and each from_bucket equals to 1')
TransitionProb.groupby(['Quarter','From_Bucket']).sum()

All the sum of transition probabilities for each quarter and each from_bucket equals to 1


To_Bucket  Probability
Quarter From_Bucket                        
2020Q1  1                   36          1.0
        2                   36          1.0
        3                   36          1.0
        4                   36          1.0
        5                   36          1.0
        6                   36          1.0
        7                   36          1.0
        8                   36          1.0
2020Q2  1                   36          1.0
        2                   36          1.0
        3                   36          1.0
        4                   36          1.0
        5                   36          1.0
        6                   36          1.0
        7                   36          1.0
        8                   36          1.0
2020Q3  1                   36          1.0
        2                   36          1.0
        3                   36          1.0
        4                   36          1.0
        5                   36          1.0
        6                   36          1.0
        7                   36          1.0
        8                   36          1.0
2020Q4  1                   36          1.0
        2                   36          1.0
        3                   36          1.0
        4                   36          1.0
        5                   36          1.0
        6                   36          1.0
        7                   36          1.0
        8                   36          1.0

# 2. After preprocessing, compute following sample statistics: 
a.	total EAD, grouped by Rating <br/>
b.	total EAD and number of unique facility by industry <br/>
c.	Find the industry which has highest averaged yield <br/>


In [11]:
# total EAD, grouped by Rating 
rating_df = df.groupby('Rating').sum()
rating_df

,Coupon,Yield,EAD,Quarter to Maturity
Rating,,,,
A1,597.155000,385.91000,1.851390e+11,1896.0
A2,1008.773226,664.52500,2.397815e+11,3424.0
A3,1228.072000,768.29702,3.444710e+11,3569.0
AA1,44.592000,34.22600,3.516200e+10,186.0
AA2,62.700000,34.60300,2.087150e+10,156.0
AA3,382.193000,245.92500,1.449295e+11,1148.0
AAA,59.630000,37.61200,2.637650e+10,224.0
BAA1,1844.218226,1224.90502,3.897135e+11,5195.0
BAA2,2106.906000,1467.61404,3.743165e+11,5861.0


In [12]:
#total EAD 
rating_df['EAD'].sum()

2074240500000.0

In [13]:
#number of unique facility by industry 
df3 = pd.read_excel("/Users/guapofish/Desktop/State_street_test.xlsx", sheet_name ="Identification")
df3.head()

,ISIN,Ticker,Name,Industry
0,US92553PBB76,TRPCN,ANR PIPELINE CO,MIDSTREAM
1,US92553PBA93,PFE,WYETH LLC,PHARMACEUTICALS
2,US349553AL15,MUNRE,MUNICH RE AMERICA CORP,P&C
3,US349553AJ68,AON,AON CORP,P&C
4,US233331AY31,BHP,BHP BILLITON FINANCE USA LTD,METALS_AND_MINING


In [14]:
# number of unique facility by industry
unique_inds = df3['Industry'].unique()
print(unique_inds)
print("The number of unique facility by industry is " + str(len(unique_inds)))

['MIDSTREAM' 'PHARMACEUTICALS' 'P&C' 'METALS_AND_MINING' 'BANKING'
 'AEROSPACE/DEFENSE' 'RAILROADS' 'HEALTH_INSURANCE' 'CONSUMER_PRODUCTS'
 'FOOD_AND_BEVERAGE' 'AUTOMOTIVE' 'PAPER' 'TECHNOLOGY'
 'BUILDING_MATERIALS' 'MEDIA_ENTERTAINMENT' 'INDEPENDENT' 'WIRELINES'
 'INTEGRATED' 'CABLE_SATELLITE' 'TRANSPORTATION_SERVICES'
 'DIVERSIFIED_MANUFACTURING' 'ENVIRONMENTAL' 'RETAILERS' 'ELECTRIC'
 'CHEMICALS' 'LIFE' 'NATURAL_GAS' 'UTILITY_OTHER' 'SUPERMARKETS'
 'OIL_FIELD_SERVICES' 'CONSTRUCTION_MACHINERY' 'WIRELESS' 'REFINING'
 'BROKERAGE_ASSETMANAGERS_EXCHANGES' 'HEALTHCARE' 'INDUSTRIAL_OTHER'
 'CONSUMER_CYCLICAL_SERVICES' 'AIRLINES' 'TOBACCO' 'RETAIL_REITS'
 'OTHER_REITS' 'OFFICE_REITS' 'RESTAURANTS' 'HEALTHCARE_REITS' 'LODGING'
 'PACKAGING' 'APARTMENT_REITS' 'FINANCE_COMPANIES' 'HOME_CONSTRUCTION'
 'FINANCIAL_OTHER' 'LEISURE']
The number of unique facility by industry is 51


In [15]:
#Find the industry which has highest averaged yield 
result = pd.merge(df, df3, how='left', on=['ISIN'])
result_y = result.groupby('Industry').mean()
result_y['Yield'].idxmax() 

'OIL_FIELD_SERVICES'

# 3. Find the industry which has lowest grades

In [16]:
df2 = pd.read_excel("/Users/guapofish/Desktop/State_street_test.xlsx", sheet_name ="Rating_to_Bucket")
result_2 = pd.merge(df2, result, how='outer', on=['Rating']).dropna()
result_2 = result_2.groupby(['Industry']).apply(lambda x: np.average(x['Bucket'],weights=x['EAD']))
print(result_2.idxmin())
result_2

UTILITY_OTHER


Industry
AEROSPACE/DEFENSE                    2.520137
AIRLINES                             2.302652
APARTMENT_REITS                      2.795634
AUTOMOTIVE                           2.753648
BANKING                              2.335554
BROKERAGE_ASSETMANAGERS_EXCHANGES    2.411156
BUILDING_MATERIALS                   2.721747
CABLE_SATELLITE                      2.480285
CHEMICALS                            2.672323
CONSTRUCTION_MACHINERY               2.020207
CONSUMER_CYCLICAL_SERVICES           2.463392
CONSUMER_PRODUCTS                    2.492780
DIVERSIFIED_MANUFACTURING            2.287634
ELECTRIC                             2.468012
ENVIRONMENTAL                        3.000000
FINANCE_COMPANIES                    2.577378
FINANCIAL_OTHER                      3.000000
FOOD_AND_BEVERAGE                    2.368488
HEALTHCARE                           2.634189
HEALTHCARE_REITS                     3.000000
HEALTH_INSURANCE                     2.319584
HOME_CONSTRUCTION        

# 4.  Write Python/R code to format the data to matrix form

-- Tab “TransitionProb” obtains the rating transition probabilities for 2020Q1 to 2020Q4; The data was extracted from a centralized database and so is saved in a non-matrix format. The desired format should be four 8*8 matrix, one for each quarter. (10 points)

In [17]:
'''
def f(x):
    return (x.pivot(index='From_Bucket', columns='To_Bucket', values='Probability'))

dff.groupby('Quarter').apply(f)

'''
matrix_4 = TransitionProb.groupby('Quarter').apply(lambda x: x.pivot(index='From_Bucket', columns='To_Bucket', values='Probability'))
matrix_4 

To_Bucket                 1         2         3         4         5         6  \
Quarter From_Bucket                                                             
2020Q1  1            0.9905  0.009500  0.000000  0.000000  0.000000  0.000000   
        2            0.0002  0.989700  0.006000  0.002000  0.001400  0.000600   
        3            0.0000  0.000983  0.965800  0.012000  0.009800  0.010197   
        4            0.0000  0.000000  0.006700  0.953500  0.021400  0.017000   
        5            0.0000  0.000000  0.000000  0.000960  0.934100  0.038700   
        6            0.0000  0.000000  0.000000  0.000002  0.000990  0.911100   
        7            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
        8            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
2020Q2  1            0.9805  0.004000  0.015500  0.000000  0.000000  0.000000   
        2            0.0002  0.979700  0.014032  0.003700  0.001600  0.000600   
        3            0.0000  0.000983  0.955800  0.012000  0.009800  0.010197   
        4            0.0000  0.000000  0.006700  0.943300  0.021400  0.017000   
        5            0.0000  0.000000  0.000000  0.000960  0.914600  0.038700   
        6            0.0000  0.000000  0.000000  0.000002  0.000990  0.901100   
        7            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
        8            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
2020Q3  1            0.9533  0.030000  0.006900  0.000770  0.000900  0.000400   
        2            0.0000  0.932300  0.040000  0.009970  0.003300  0.002200   
        3            0.0000  0.000000  0.930000  0.022000  0.009800  0.010197   
        4            0.0000  0.000000  0.009067  0.882100  0.034900  0.027000   
        5            0.0000  0.000000  0.000000  0.000000  0.880000  0.038700   
        6            0.0000  0.000000  0.000000  0.000000  0.000034  0.878700   
        7            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
        8            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
2020Q4  1            0.9677  0.020000  0.006900  0.000770  0.000900  0.000400   
        2            0.0000  0.952400  0.040000  0.009970  0.003300  0.002200   
        3            0.0000  0.000000  0.941100  0.022000  0.009800  0.010197   
        4            0.0000  0.000000  0.009067  0.908900  0.047900  0.004300   
        5            0.0000  0.000000  0.000000  0.000000  0.902100  0.054000   
        6            0.0000  0.000000  0.000000  0.000000  0.000000  0.880000   
        7            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
        8            0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   

To_Bucket                   7         8  
Quarter From_Bucket                      
2020Q1  1            0.000000  0.000000  
        2            0.000100  0.000000  
        3            0.000670  0.000550  
        4            0.000760  0.000640  
        5            0.015500  0.010740  
        6            0.059280  0.028628  
        7            0.901600  0.098400  
        8            0.000000  1.000000  
2020Q2  1            0.000000  0.000000  
        2            0.000100  0.000068  
        3            0.010670  0.000550  
        4            0.002760  0.008840  
        5            0.020000  0.025740  
        6            0.067280  0.030628  
        7            0.880100  0.119900  
        8            0.000000  1.000000  
2020Q3  1            0.000067  0.007663  
        2            0.000059  0.012171  
        3            0.010670  0.017333  
        4            0.006400  0.040533  
        5            0.020000  0.061300  
        6            0.008892  0.112374  
        7            0.850000  0.150000  
        8            0.000000  1.000000  
2020Q4  1            0.000067  0.003263  
        2            0.000059 -0.007929  
        3            0.010670  0.006233  
        4            0.0064

# 5.	For each facility, compute cumulative default probability from Dec 31, 2019 (end of 2019Q4) to maturity date 

In [18]:
#average transition matrix after one year
matrix = matrix_4.reset_index().groupby('From_Bucket').mean()
matrix

To_Bucket,1,2,3,4,5,6,7,8
From_Bucket,,,,,,,,
1,0.9730,0.015875,0.007325,0.000385,0.000450,0.000200,0.000034,0.002731
2,0.0001,0.963525,0.025008,0.006410,0.002400,0.001400,0.000080,0.001077
3,0.0000,0.000491,0.948175,0.017000,0.009800,0.010197,0.008170,0.006166
4,0.0000,0.000000,0.007883,0.921950,0.031400,0.016325,0.004080,0.018361
5,0.0000,0.000000,0.000000,0.000480,0.907700,0.042525,0.022125,0.027170
6,0.0000,0.000000,0.000000,0.000001,0.000504,0.892725,0.036336,0.070434
7,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.864425,0.135575
8,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [19]:
f = pd.merge(df2, result, how='outer', on=['Rating']).dropna()
f['Quarter to Maturity'] = f['Quarter to Maturity'].astype(int)
f['default_probability'] = 0  #initiaize a new column

In [20]:
f.head()

,Rating,Bucket,ISIN,Coupon,Maturity Date,Yield,EAD,Quarter to Maturity,Ticker,Name,Industry,default_probability
0,AAA,1,US001192AH64,2.700,2025-02-12,2.278,2.291000e+09,21,MSFT,MICROSOFT CORP,TECHNOLOGY,0
1,AAA,1,US023551AJ38,1.550,2021-08-08,1.633,2.699500e+09,6,MSFT,MICROSOFT CORP,TECHNOLOGY,0
2,AAA,1,US02364WAP05,2.050,2023-03-01,1.730,5.020000e+08,13,JNJ,JOHNSON & JOHNSON,PHARMACEUTICALS,0
3,AAA,1,US03523TBE73,2.375,2022-02-12,1.792,1.523500e+09,9,MSFT,MICROSOFT CORP,TECHNOLOGY,0
4,AAA,1,US03523TBF49,1.850,2020-02-12,1.385,1.501000e+09,0,MSFT,MICROSOFT CORP,TECHNOLOGY,0


In [21]:
matrix_term = []
matrix_term.append(matrix_4.loc['2020Q1'])
matrix_term.append(matrix_4.loc['2020Q2'])
matrix_term.append(matrix_4.loc['2020Q3'])
matrix_term.append(matrix_4.loc['2020Q4'])
max_period = int(f['Quarter to Maturity'].max())

for i in range(max_period-4):
    matrix_term.append(matrix)

# len(matrix_term) = 24 good
# Cumlulative default probaility for each time Quarter to Maturity item

for i in range(max_period-1):
    matrix_term[i+1] = matrix_term[i+1]*matrix_term[i]

print(matrix_term)

[To_Bucket         1         2       3         4        5         6        7  \
From_Bucket                                                                   
1            0.9905  0.009500  0.0000  0.000000  0.00000  0.000000  0.00000   
2            0.0002  0.989700  0.0060  0.002000  0.00140  0.000600  0.00010   
3            0.0000  0.000983  0.9658  0.012000  0.00980  0.010197  0.00067   
4            0.0000  0.000000  0.0067  0.953500  0.02140  0.017000  0.00076   
5            0.0000  0.000000  0.0000  0.000960  0.93410  0.038700  0.01550   
6            0.0000  0.000000  0.0000  0.000002  0.00099  0.911100  0.05928   
7            0.0000  0.000000  0.0000  0.000000  0.00000  0.000000  0.90160   
8            0.0000  0.000000  0.0000  0.000000  0.00000  0.000000  0.00000   

To_Bucket           8  
From_Bucket            
1            0.000000  
2            0.000000  
3            0.000550  
4            0.000640  
5            0.010740  
6            0.028628  
7            0.0

In [22]:
num = len(f)
# f['default_probaility'][0] = matrix_term[f['Quarter to Maturity'][0]].loc[f['Bucket'][0]][8]

for i in range(num+1):    
    try:
        f['default_probability'].loc[i] = matrix_term[f['Quarter to Maturity'][i]-1].loc[f['Bucket'][i]][8]    
    except:
        # f['default_probaility'][1073] error   
        continue    

/Users/guapofish/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
df = f[['Rating','Quarter to Maturity','default_probability']]
df

,Rating,Quarter to Maturity,default_probability
0,AAA,21,0.000000e+00
1,AAA,6,0.000000e+00
2,AAA,13,0.000000e+00
3,AAA,9,0.000000e+00
4,AAA,0,0.000000e+00
...,...,...,...
2462,BAA1,22,5.432422e-51
2463,BAA1,21,8.809571e-49
2464,BAA1,16,9.880094e-38
2465,BAA1,16,9.880094e-38


In [24]:
df.groupby(['Quarter to Maturity','Rating']).mean()

default_probability
Quarter to Maturity Rating                     
0                   A1             0.000000e+00
                    A2             0.000000e+00
                    A3             0.000000e+00
                    AA1            0.000000e+00
                    AA3            0.000000e+00
...                                         ...
24                  A3             0.000000e+00
                    AA3            0.000000e+00
                    BAA1           2.065718e-55
                    BAA2           2.065718e-55
                    BAA3           2.065718e-55

[210 rows x 1 columns]

In [25]:
f['default_probability'].max()

0.00055